# Magic Personality Matcher - Preprocesamiento de datos

Para evitar tener que preprocesar los datos cada vez que abrimos el Google Colab vamos a realizar tanto particionado de los datos como su preprocesamiento en este notebook aparte.

## Fase 1 - Cargar el dataset

Lo primero es cargar el dataset con Pandas y seleccionar la cantidad de filas que queremos usar:

In [23]:
import pandas as pd

raw_data = pd.read_csv("dataset_definitivo.csv")
print("general counts")
print(raw_data['personality'].value_counts())
training_data = raw_data#.head(5000)

print("used dataset info")
print(training_data.shape)
print(training_data.head())
print(training_data['personality'].value_counts())

general counts
personality
INTP    27076
INTJ    24299
INFJ    17490
INFP    15248
ENTP    12987
ENFP     7571
ISTP     4088
ENTJ     3465
ENFJ     2242
ESTP     2175
ISTJ     1707
ISFP     1513
ISFJ     1180
ESTJ      602
ESFP      582
ESFJ      328
Name: count, dtype: int64
used dataset info
(122553, 2)
  personality                                               post
0        INTJ  @Pericles216 @HierBeforeTheAC @Sachinettiyil T...
1        INTJ  @Hispanthicckk Being you makes you look cute||...
2        INTJ  @Alshymi Les balles sont réelles et sont tirée...
3        INTJ  I'm like entp but idiotic|||Hey boy, do you wa...
4        INTJ  @kaeshurr1 Give it to @ZargarShanif ... He has...
personality
INTP    27076
INTJ    24299
INFJ    17490
INFP    15248
ENTP    12987
ENFP     7571
ISTP     4088
ENTJ     3465
ENFJ     2242
ESTP     2175
ISTJ     1707
ISFP     1513
ISFJ     1180
ESTJ      602
ESFP      582
ESFJ      328
Name: count, dtype: int64


## Fase 2 - Filtrado de Stopwords, Tokenización y Stemming

El preprocesamiento de texto se realiza usando las stopwords y el tokenizer de la librería de lenguaje natural de Python. Primero definimos las stopwords, que serán las de la librería de Python y las personalidades Myers-Briggs para evitar introducir sesgos, como que, si se menciona una personalidad, que esta no se tenga en cuenta para decidir la del autor. Luego se usa el PorterStemmer para obtener la “raíz” de las palabras. Entonces el resultado es una nueva columna que contiene los posts preprocesados.

In [ ]:
import multiprocessing
#from tkinter.constants import W
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm.contrib.concurrent import process_map
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()


STOPS = set(stopwords.words("english"))
CUSTOM_STOPS=set(['istj', 'isfj', 'infj', 'intj', 'istp', 'isfp', 'infp', 'intp', 'estp', 'esfp', 'enfp', 'entp', 'estj', 'esfj', 'enfj', 'entj'])

def process_text(post):
    post = post.lower()
    post = re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',post)
    post = re.sub('[^0-9a-z]',' ',post)
    return " ".join([ps.stem(w) for w in word_tokenize(post) if not w in STOPS and w not in CUSTOM_STOPS])

preprocessedData = training_data.loc[:]

num_processes = multiprocessing.cpu_count()

try:
    with multiprocessing.Pool(processes=num_processes) as pool:
        preprocessedData['processed_text'] = process_map(process_text, training_data['post'], max_workers=num_processes)
finally:
    pool.close()
    pool.join()

preprocessedData

[nltk_data] Downloading package punkt to /home/mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_8206/860219597.py:31: TqdmWarning: Iterable length 122553 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  preprocessedData['processed_text'] = process_map(process_text, training_data['post'], max_workers=num_processes)
 10%|███▋                               | 12734/122553 [00:44<04:38, 393.89it/s]

## Fase 3 - Particionar el dataset para entrenamiento y validación
Se divide el dataset en 2 para tener 80% datos de entrenamiento y 20% de prueba

In [18]:
from sklearn.model_selection import train_test_split

X = preprocessedData['processed_text']
Y = preprocessedData['personality']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Fase 4 - Bolsa de palabras

Se crea una bolsa de palabras utilizando TfidVectorizer, que se basa en la frecuencia de las palabras para determinar su importancia.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Datos de entrenamiento
bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(X_train)
train_textsBoW = bagOfWordsModel.transform(X_train)
print("X_train bag of words:")
print(train_textsBoW.shape)

# Datos pruebas
bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(X_test)
test_textsBoW = bagOfWordsModel.transform(X_test)
print("X_test bag of words:")
print(test_textsBoW.shape)

X_train bag of words:
(4000, 207345)
X_test bag of words:
(1000, 76462)


## Fase 5 - Guardar el resultado

Utilizamos joblib para guardar estas bolsas de palabras para poder usarlas para entrenamiento sin tener que repetir el proceso.

In [22]:
from joblib import dump

dump(train_textsBoW, "train_textsBoW.joblib")
dump(test_textsBoW, "test_textsBoW.joblib")

['test_textsBoW.joblib']